# TP -- Pracical work

### Table of Contents

* [Introduction](#introduction)
* [1. Analysing variability, capacity factors](#1.variability)
    * [Q1.1. Load factor (production/consumption)](#1.1.Question)
    * [Q1.2. Most simple dimensioning rules -- Lower and upper bound](#1.2.Question)
    * [Q1.3. Dimensioning -- Adding a storage](#1.3.Question)
* [2. Economic analysis, computing LCOE, system LCOE](#2.LCOE)
    * [Q2.1. LCOE of renewable + Flex system](#2.1.Question)
* [3. Optimisation of operation](#3.Operation)
    * [Q3.1. Optimisation results and Lagrange multipliers](#3.1.Question)
    * [Q3.2. Storage optimisation](#3.2.Question)
* [4. Optimisation of planning](#4.Planning)
    * [Q4.1. Optimisation results and Lagrange multipliers](#4.1.Question)



## Introduction <a class="anchor" id="introduction"></a>

Question asked here can generally be answered by copy-past of code given in folder "BasicFunctionalities" + small modifications/additions of yoru own.


In [1]:
import os
if os.path.basename(os.getcwd())=="SujetsDAnalyses":
    os.chdir('..') ## to work at project root  like in any IDE
if os.path.basename(os.getcwd())=="Selected":
    os.chdir('../..') ## to work at project root  like in any IDE
InputFolder='Data/input/'

#region importation of modules
import numpy as np
import pandas as pd
import seaborn as sns
import csv
import datetime
import copy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn import linear_model
from functions.f_operationModels import *
from functions.f_optimization import *
from functions.f_graphicalTools import *
import sys

#endregion

#region Solver and data location definition

InputFolder='Data/input/'

if sys.platform != 'win32':
    myhost = os.uname()[1]
else : myhost = ""
if (myhost=="jupyter-sop"):
    ## for https://jupyter-sop.mines-paristech.fr/ users, you need to
    #  (1) run the following to loanch the license server
    if (os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmgrd -c /opt/mosek/9.2/tools/platform/linux64x86/bin/mosek.lic -l lmgrd.log")==0):
        os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmutil lmstat -c 27007@127.0.0.1 -a")
    #  (2) definition of license
    os.environ["MOSEKLM_LICENSE_FILE"] = '@jupyter-sop'

BaseSolverPath='/Users/robin.girard/Documents/Code/Packages/solvers/ampl_macosx64' ### change this to the folder with knitro ampl ...
## in order to obtain more solver see see https://ampl.com/products/solvers/open-source/
## for eduction this site provides also several professional solvers, that are more efficient than e.g. cbc
sys.path.append(BaseSolverPath)
solvers= ['gurobi','knitro','cbc'] # try 'glpk', 'cplex'
solverpath= {}
for solver in solvers : solverpath[solver]=BaseSolverPath+'/'+solver
solver= 'mosek' ## no need for solverpath with mosek.
#endregion


## 1. Analysing variability, capacity factors  <a class="anchor" id="1.variability"></a>
To answer questions here, you only need to understand thermal sensitivity manipulations explained in file BasicFunctionalities/input-Consumption.ipynb and
and know how to extract Availability data here : BasicFunctionalities/input-Availability.ipynb. You have to know the definition of capacity factor,
and the difference between installed power and energy produced. You can find the corresponding definitions in the first two sections of
the post [here](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).

### Q1.1. Load factor (production/consumption)  <a class="anchor" id="1.1.Question"></a>

**Load factor of consumption**

From the available data, compute load factor of consumption mean($C_t$)/max($C_t$) [%]
for

 - year 2016 for different countries available,
 - 2013 à 2017 de 0 à 2,5 GW/°C


In [2]:
from functions.f_consumptionModels import *
#region multi zone
area="FR_DE_GB_ES"
year=2016 #only possible year
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(area)+'.csv',
                                sep=',',decimal='.',skiprows=0).set_index(["AREAS","TIMESTAMP"])
#print(areaConsumption)
for region in ["FR","DE","ES","GB"]: ### problem with spain data
    conso = areaConsumption.loc[(region,slice(None)),"areaConsumption"]
    mean = conso.mean()
    maxi = conso.max()
    print('Facteur de charge de la région : ',region,' : ',"%.4f" % (mean/maxi))


print('###################')

ConsoTempe_df=pd.read_csv(InputFolder+'ConsumptionTemperature_1996TO2019_FR.csv')
ConsoTempe_df["TIMESTAMP"]=pd.to_datetime(ConsoTempe_df['Date'])
ConsoTempe_df=ConsoTempe_df.drop(columns=["Date"]).set_index(["TIMESTAMP"])

for year in [2013,2014,2015,2016,2017]:
    ConsoTempeYear_df=ConsoTempe_df[str(year)]
    mean = ConsoTempeYear_df['Consumption'].mean()
    maxi = ConsoTempeYear_df['Consumption'].max()
    print('Facteur de charge en France pour l\'année {} : '.format(year),"%.3f" % (mean/maxi))
    (ConsoTempeYear_decomposed_df,Thermosensibilite)= Decomposeconso(ConsoTempeYear_df,TemperatureThreshold=15)
    #print(Thermosensibilite)
    for thermo_sens in [0,1,1.5,2]:
        NewThermosensibilite={}
        for key in Thermosensibilite:    NewThermosensibilite[key]= -thermo_sens*1000
        NewConsoTempeYear_decomposed_df=Recompose(ConsoTempeYear_decomposed_df,NewThermosensibilite,
                                                  TemperatureThreshold=15)
        mean = NewConsoTempeYear_decomposed_df['Consumption'].mean()
        maxi = NewConsoTempeYear_decomposed_df['Consumption'].max()
        print('Avec {:.1f} GW/°C : '.format(thermo_sens),"%.3f" %(mean/maxi))

Facteur de charge de la région :  FR  :  0.6162
Facteur de charge de la région :  DE  :  0.7266
Facteur de charge de la région :  ES  :  0.7101
Facteur de charge de la région :  GB  :  0.6066
###################
Facteur de charge en France pour l'année 2013 :  0.607
Avec 0.0 GW/°C :  0.663
Avec 1.0 GW/°C :  0.682
Avec 1.5 GW/°C :  0.657
Avec 2.0 GW/°C :  0.621
Facteur de charge en France pour l'année 2014 :  0.639
Avec 0.0 GW/°C :  0.644
Avec 1.0 GW/°C :  0.660
Avec 1.5 GW/°C :  0.647
Avec 2.0 GW/°C :  0.628
Facteur de charge en France pour l'année 2015 :  0.587
Avec 0.0 GW/°C :  0.675
Avec 1.0 GW/°C :  0.649
Avec 1.5 GW/°C :  0.619
Avec 2.0 GW/°C :  0.592
Facteur de charge en France pour l'année 2016 :  0.616
Avec 0.0 GW/°C :  0.710
Avec 1.0 GW/°C :  0.695
Avec 1.5 GW/°C :  0.670
Avec 2.0 GW/°C :  0.641
Facteur de charge en France pour l'année 2017 :  0.580
Avec 0.0 GW/°C :  0.657
Avec 1.0 GW/°C :  0.648
Avec 1.5 GW/°C :  0.631
Avec 2.0 GW/°C :  0.604


**Load factor of Wind Power and PV production**

Compute load factor of renewable production (PV/Wind) for different years/countries that are available (see input-Availability.ipynb).

In [3]:
for tech in ['WindOnShore','Solar']:#'Thermal' 'OldNuke' 'HydroRiver' 'HydroReservoir' 'WindOnShore' 'Solar'
    print('######################################################')
    print('Load factor of {} technology'.format(tech))
    for area in ["FR"]:
        print('in France: ')
        for year in range(2013,2017):
            availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(area)+'.csv',
                                sep=',',decimal='.',skiprows=0)
            tabl=availabilityFactor[availabilityFactor['TECHNOLOGIES']==tech]
            print('{} :'.format(year),"%.3f" % (tabl['availabilityFactor'].mean()))
print('######################################################')
print('######################################################')

print('In 2016 Europa')
#import data
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor2016_FR_DE_GB_ES.csv'
                                 ,sep=',',decimal='.',skiprows=0)

for tech in ['WindOnShore','WindOffShore','Solar']:#'Thermal' 'OldNuke' 'HydroRiver' 'HydroReservoir' 'WindOnShore' 'Solar'
    print('Load factor of {} technology'.format(tech))
    for area in ["FR","DE","ES","GB"]:
        tabl=availabilityFactor[availabilityFactor['TECHNOLOGIES']==tech]
        tabl=tabl[tabl['AREAS']==area]
        print("{} : ".format(area),"%.3f" % (tabl['availabilityFactor'].mean()))
    print('######################################################')

######################################################
Load factor of WindOnShore technology
in France: 
2013 : 0.233
2014 : 0.229
2015 : 0.248
2016 : 0.220
######################################################
Load factor of Solar technology
in France: 
2013 : 0.135
2014 : 0.143
2015 : 0.148
2016 : 0.144
######################################################
######################################################
In 2016 Europa
Load factor of WindOnShore technology
FR :  0.220
DE :  0.176
ES :  0.235
GB :  0.246
######################################################
Load factor of WindOffShore technology
FR :  0.279
DE :  0.228
ES :  0.303
GB :  0.312
######################################################
Load factor of Solar technology
FR :  0.144
DE :  0.099
ES :  0.123
GB :  0.221
######################################################


**"Load factor" of nuclear availability**

Compute the "load factor" of nuclear availability for year 2007 to 2016.

In [4]:
#DispoNukeTotal2007_2017.csv
nukeConso = pd.read_csv(InputFolder+'DispoNukeTotal2007_2017.csv',
                                sep=';',decimal=',',skiprows=0,dtype={'Availability':np.float64})

print('Nuclear load factor in France')
for year in range(2007, 2017):
            nukeConso['year'] = pd.DatetimeIndex(nukeConso['Dates']).year
            tabl=nukeConso[nukeConso['year']==year]
            print('in year {} :'.format(year),"%.5f" % (tabl['Availability'].mean()/tabl['Availability'].max()))
print('no data for 2017')


Nuclear load factor in France
in year 2007 : 0.84967
in year 2008 : 0.82094
in year 2009 : 0.78128
in year 2010 : 0.81332
in year 2011 : 0.80337
in year 2012 : 0.79290
in year 2013 : 0.80118
in year 2014 : 0.81614
in year 2015 : 0.81357
in year 2016 : 0.75576
no data for 2017



### Q1.2. Most simple dimensioning rules -- Lower and upper bound  <a class="anchor" id="1.2.Question"></a>

 **a -- Lower bound**

How much power of XXX power would you need to install in LLL to have an **annual energy** produced equal to the annual energy consummed in year YYY ?
 Answer this question for XXX = {Wind, PV, Nuke}, LLL =available country and YYY available year.

In [5]:
year = 2016
area = 'FR'
consumption_table  = {}

for year in range(2013,2017):

    consumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(area)+'.csv',
                                    sep=',',decimal='.',skiprows=0)
    energyConso = consumption['areaConsumption'].sum()
    consumption_table[year] = energyConso
    print('total consumption of {} :'.format(year),"%.3f" % (energyConso/1e6),'TWh')

print('######################################################')
print('######################################################')

print('Mean Load Factor per sector')
for tech in ['WindOnShore','Solar','OldNuke']:
    print('######################################################')
    for year in range(2013,2017):
        availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(area)+'.csv',
                            sep=',',decimal='.',skiprows=0)
        tabl=availabilityFactor[availabilityFactor['TECHNOLOGIES']==tech]
        print(year," ",tech," : ","%.3f" % (tabl['availabilityFactor'].mean()))
        print('\t\t Lower bound : ',"%.3f" % ((consumption_table[year]/tabl['availabilityFactor'].mean())/(8760*1e3)),'GW')


total consumption of 2013 : 492.015 TWh
total consumption of 2014 : 462.467 TWh
total consumption of 2015 : 473.104 TWh
total consumption of 2016 : 480.291 TWh
######################################################
######################################################
Mean Load Factor per sector
######################################################
2013   WindOnShore  :  0.233
		 Lower bound :  240.965 GW
2014   WindOnShore  :  0.229
		 Lower bound :  230.225 GW
2015   WindOnShore  :  0.248
		 Lower bound :  217.762 GW
2016   WindOnShore  :  0.220
		 Lower bound :  248.872 GW
######################################################
2013   Solar  :  0.135
		 Lower bound :  416.965 GW
2014   Solar  :  0.143
		 Lower bound :  369.209 GW
2015   Solar  :  0.148
		 Lower bound :  364.830 GW
2016   Solar  :  0.144
		 Lower bound :  380.086 GW
######################################################
2013   OldNuke  :  0.759
		 Lower bound :  73.979 GW
2014   OldNuke  :  0.791
		 Lower bound :  6

 **b -- Upper bound**

How much power of XXX power would you need to install in LLL to have an **hourly energy** produced equal to the hourly energy consummed in year YYY ?
 Answer this question for XXX = {Wind, Nuke}, LLL =available country and YYY available year.
 Assuming that for all hours of the year the excess of production is lost, what would be the capacity factor of productions in this cases

In [6]:
area="FR"

print('######################################################')
print('######################################################')

print('Mean Load Factor per sector')
for tech in ['WindOnShore','OldNuke']:
    print('######################################################')
    for year in range(2013,2017) : #range(2013,2017):
        availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(area)+'.csv',
                            sep=',',decimal='.',skiprows=0).set_index(['TIMESTAMP','TECHNOLOGIES'])
        consumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(area)+'.csv',
                                    sep=',',decimal='.',skiprows=0).set_index(['TIMESTAMP'])
        #print(consumption['areaConsumption'])
        AvailabilityTech=availabilityFactor.loc[(slice(None),str(tech)),:].reset_index().set_index("TIMESTAMP").drop(["TECHNOLOGIES"], axis=1)

        print(year," ",tech," : ")
        print('\t\t upper bound : ',"%.3f" % ( (consumption.areaConsumption/AvailabilityTech.availabilityFactor).max()/1e3),'GW')




######################################################
######################################################
Mean Load Factor per sector
######################################################
2013   WindOnShore  : 
		 upper bound :  9007.383 GW
2014   WindOnShore  : 
		 upper bound :  16169.042 GW
2015   WindOnShore  : 
		 upper bound :  24926.397 GW
2016   WindOnShore  : 
		 upper bound :  13449.137 GW
######################################################
2013   OldNuke  : 
		 upper bound :  106.410 GW
2014   OldNuke  : 
		 upper bound :  94.064 GW
2015   OldNuke  : 
		 upper bound :  98.030 GW
2016   OldNuke  : 
		 upper bound :  113.853 GW


 **c -- Sensitivity of the result to thermal sensitivity of consumption**

 For year 2016 and in France, analyse the sensitivity of the preceding results (upper bound and lower bound) on the thermal sensitivity of consumption

In [7]:
# Put your answer here


### Q1.3 Dimensioning -- Adding a storage <a class="anchor" id="1.3.Question"></a>
These questions are much more difficult than the preceding ones. It is possible that no student will manage to do it during the practical work session.

**a - Dimensioning a system with Renewable + "infinite storage"**

We now assume we have a theoretical storage with
 - infinite energy capacity (can store for hours without being full start with an initial capacity to discuss)
 - a maximum production power (of the storage) that will be equal to the maximum consumption power (of the storage)
 - These maximum power are supposed ajusted so that $C_t\leq P_t+S_t$ for all hours $t$ of the considered year
 (With $C$ the consumption,$P$ the wind power production, $S$ the storage production. $S$ can be negative when it is a consumption)
 - an efficiency $\eta=70%$



Try to compute :
 - installed power of wind and storage,
 - losses of wind energy through the storage
 - losses of wind energy that cannot be stored of consumed directly.

In [8]:
# Put your answer here


**b -adding french hydro power**

 In the preceding system, you can add french hydro production (the data can be found in Eco2mix section of input-Availability.ipynb)

In [9]:
# Put your answer here

**c -sensitibvity to thermal sensitivity**

 What is the sensitivity of the preceding results to thermal sensitivity of consumption

In [10]:
# Put your answer here

** d -Adding PV **
In the preceding system, assume now you can mix PV and wind. What can you gain ?

In [11]:
# Put your answer here


If you finish all this before the end of the first session, of if you wish to dig a little bit more, you can read [this article](https://jancovici.com/transition-energetique/renouvelables/100-renouvelable-pour-pas-plus-cher-fastoche/)
and discuss the dimensioning choice for the storage that is proposed there. Otherwise, or after that, you can switch to question 2.

## 2. Economic analysis, computing LCOE, system LCOE <a class="anchor" id="2.LCOE"></a>

### Q2.1. LCOE of renewable + Flex system <a class="anchor" id="2.1.Question"></a>
Read and understand jupyter notebook optim-Operation.ipynb.
Try to imagine systems that are feasible and compute the corresponding system LCOE in the two following cases:
- Nuke + CCG + hydro
- Renewable + CCG + hydro
You will need to set the installed power and produced energy for each of these production mean. Try to get inspiration from preceding section.
There are no "right answer", the idea here is to have rough estimates that will be improved later.
Try adding significant amount of electrolyser/fuel cells to replace CCG. How much capacity would you install and see how it changes the results ?
Try changing the thermal sensitivity of consumption.
You can also analyse the effect of discount rate, or carbon tax, or cost of gaz (by assumign e.g. it is a very expensive biogaz).

In [12]:
# Put your answer here

## 3. Optimisation of operation <a class="anchor" id="3.Operation"></a>

### Q3.1. Optimisation results and Lagrange multipliers <a class="anchor" id="3.1.Question"></a>
Read and understand jupyter notebook optim-Operation.ipynb. Can you explain the value of all lagrange multiplier in a simple case with two production means ?
With one area and then with two. How is it possible to generate negative lagrange multipliers ? Propose a simple case where they appear and explain the corresponding values.


In [13]:
# Put your answer here

### Q3.2. Storage optimisation <a class="anchor" id="3.2.Question"></a>
Read and understand jupyter notebook optim-storage.ipynb. What is the maximal price for the storage system to be profitable today ?
Apply an homothetic transformation to increase the variance of the prices but without changing the mean (and by keeping positive prices).
How does it change the preceding results ?

In [14]:
# Put your answer here


## 4. Optimisation of planning <a class="anchor" id="4.Planning"></a>

### Q4.1. Optimisation results and Lagrange multipliers <a class="anchor" id="4.1.Question"></a>

**a - System LCOE**

Read and understand jupyter notebook optim-storage.ipynb. Try using it to adapt your answer to Q2.1.

In [15]:
# Put your answer here

**a - Loss of load expectation**

Today the expected loss of load is of 3 hours in average per year, try to find how to set the variable cost of "curtailment"
(consumption that is cut, the cost is initially set to 3000 €/MWh) to have the value of this expected number at 3.1.
Try to find in the CRE or RTE or ENEDIS website the "official value" in France.

In [16]:
# Put your answer here